In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from proofreader.utils.vis import plot_3d
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from proofreader.data.cremi import prepare_cremi_vols

train_vols, test_vols = prepare_cremi_vols('../../dataset/cremi')

In [ ]:
from proofreader.data.splitter import NeuriteDataset
from proofreader.data.augment import Augmentor
num_slices = [4, 4]
radius = 96
context_slices = 4
num_points = 1024

augmentor = Augmentor(center=True, shuffle=True, normalize=[125, 1250, 1250])
train_dataset = NeuriteDataset(test_vols, num_slices, radius, context_slices, num_points=num_points, torch=True, open_vol=True, verbose=False, Augmentor=augmentor)
print(len(train_dataset))

In [ ]:
from torch.utils.data import DataLoader
import torch.nn as nn
config = get_config('cn_context_4_aug_small')
dataloader = DataLoader(dataset=train_dataset, batch_size=2, shuffle=True)
model, _ ,_ = build_full_model_from_config(config.model, config.dataset)
model = nn.DataParallel(model)
model = load_model(model, '../../330.ckpt', map_location=torch.device('cpu'))



In [ ]:
from proofreader.model.classifier import *
with torch.no_grad():
    count, acc = 0,0
    for step, batch in enumerate(dataloader):
        count += 1
        # get batch
        x, y = batch
        y_hat = model(x)
        pred = predict_class(y_hat)
        accs = get_accuracy(y, pred)
        print(accs)
        acc += accs['total_acc']
        print(round(acc/count, 3))


In [80]:
def plot_example(x,y):
    lim = (-.1,.1)
    lim = (lim,lim,lim)
    pc = np.swapaxes(x, 0, 1)
    label = y.item() == 1
    plot_3d(pc, title=label, lims=lim)


In [121]:
from proofreader.data.augment import Augmentor
from proofreader.utils.torch import load_model
from proofreader.model.config import *
from proofreader.model.classifier import *
from torch.utils.data import DataLoader
import torch.nn as nn

config = get_config('pointnet-pre-cs2')
ds_config = config.dataset

model = PointNet(num_points=2000, classes=2, batch_norm=True)
model = nn.DataParallel(model)
model = load_model(model, '../../750.ckpt', map_location=torch.device('cpu'))

augmentor = Augmentor(center=True, shuffle=True, normalize=[125, 1250, 1250])
num_slices = 1
tester = SliceDataset(test_vols, num_slices, 96, 2, 
                    num_points=2000, Augmentor=augmentor, verbose=False, drop_false=True, return_candidate_batch=True, randomize=True)


In [122]:

with torch.no_grad():
    model.eval()
    neurites = 0
    seen = 0
    seen_correct = 0
    neurite_correct = 0
    for step, batch in enumerate(tester):
      
        # get batch
        x, y = batch

        for i in range(x.shape[0]): 
            y_hat = model(x[i].unsqueeze(dim=0))
            pred = predict_class(y_hat)
            true = int(pred == y[i])
            seen_correct += true
            seen += 1
            
            if pred == 1 or y[i] == 1:
                neurites += 1
                neurite_correct += true
                # if true == 0:
                #     plot_example(x[i],y[i])
                break

        if step % 10 == 0:
            print('seen acc:', seen_correct/seen, 'neurite acc:', neurite_correct/neurites)
            print(tester.get_stats())
        if step > 1000:
            break

print('seen acc:', seen_correct/seen)
print('neurite acc:', neurite_correct/neurites)
print(tester.get_stats())


random drop start 8 vol 0
seen acc: 1.0 neurite acc: 1.0
no_true: 0, multi_true: 0, num_true: 1, total: 1)
seen acc: 0.9166666666666666 neurite acc: 0.9090909090909091
no_true: 0, multi_true: 0, num_true: 11, total: 13)
seen acc: 0.9565217391304348 neurite acc: 0.9523809523809523
no_true: 0, multi_true: 1, num_true: 21, total: 24)
seen acc: 0.9696969696969697 neurite acc: 0.967741935483871
no_true: 0, multi_true: 1, num_true: 31, total: 34)
seen acc: 0.9545454545454546 neurite acc: 0.9512195121951219
no_true: 0, multi_true: 1, num_true: 41, total: 46)
seen acc: 0.9636363636363636 neurite acc: 0.9607843137254902
no_true: 0, multi_true: 1, num_true: 51, total: 57)
seen acc: 0.9538461538461539 neurite acc: 0.9508196721311475
no_true: 0, multi_true: 2, num_true: 61, total: 68)
seen acc: 0.96 neurite acc: 0.9577464788732394
no_true: 0, multi_true: 2, num_true: 71, total: 78)
seen acc: 0.9651162790697675 neurite acc: 0.9629629629629629
no_true: 0, multi_true: 2, num_true: 81, total: 89)
seen